# CTE - Common Table Expression

## Allgemeines

\* eingeführt mit SQL Server 2015

## recursive CTE

Eine rekursive CTE ist eine CTE, die sich selbst referenziert. Hierbei wird die CTE wiederholt ausgeführt bis das ResultSet fertig ist.

![SQL Server Recursive CTE execution flow](https://www.sqlservertutorial.net/wp-content/uploads/SQL-Server-Recursive-CTE-execution-flow.png)

In [ ]:
-- Wochentage
WITH cte_numbers(n, weekday) 
AS (
    SELECT 
        0, 
        DATENAME(DW, 0)
    UNION ALL
    SELECT    
        n + 1, 
        DATENAME(DW, n + 1)
    FROM    
        cte_numbers
    WHERE n < 6
)
SELECT 
    weekday
FROM 
    cte_numbers;

In [ ]:
-- hundert Tage ab heute
WITH datelist(n,[Date]) 
AS (
    SELECT 
        0, DateAdd(day,0,getdate()) as [Date]
    UNION ALL
    SELECT    
        n + 1, DateAdd(day,n + 1,getdate()) as [Date]
    FROM    
        datelist
    WHERE n < 100
)
SELECT 
    n, [Date]
FROM 
    datelist;

Für Hierachien:

In [ ]:
-- Rohdaten:
USE [OLTP_Northwind]
GO
SELECT [EmployeeID]
      ,[LastName]
      ,[FirstName]
      ,[ReportsTo]
  FROM [OLTP_Northwind].[dbo].[Employees]

In [ ]:
USE [OLTP_Northwind]
GO
;With CTE_Org as (
Select [EmployeeID], [LastName] + ' ' + [FirstName] as [EmployeeName],  [ReportsTo]from [dbo].[Employees] where [ReportsTo] is null
union all
Select e.[EmployeeID], e.[LastName] + ' ' + e.[FirstName]  as [ManagerName], e.[ReportsTo] from [dbo].[Employees]  as e
inner join CTE_Org as o on o.EmployeeId = e.[ReportsTo])

Select * from CTE_Org

In [ ]:
-- mit Limitation (Execution Hint): Zeige mir die zwei höchsten Ebenen des Unternehmens.
USE [OLTP_Northwind]
GO
;With CTE_Org as (
Select [EmployeeID], [LastName] + ' ' + [FirstName] as [EmployeeName],  [ReportsTo]from [dbo].[Employees] where [ReportsTo] is null
union all
Select e.[EmployeeID], e.[LastName] + ' ' + e.[FirstName]  as [ManagerName], e.[ReportsTo] from [dbo].[Employees]  as e
inner join CTE_Org as o on o.EmployeeId = e.[ReportsTo])

Select * from CTE_Org OPTION (MAXRECURSION 2) --dh. nur 2 Ebenen

Fortlaufende summen / Consecutive Sum / Running Total

In [ ]:
WITH  consecutive_number_sum (i, consecutive_sum) AS (
  SELECT 0, 0
  UNION ALL
  SELECT i + 1, (i + 1) + consecutive_sum
  FROM consecutive_number_sum
  WHERE i < 50
)
SELECT i, consecutive_sum
FROM consecutive_number_sum

## Wiederverwendbarkeit von CTE

In [4]:
USE [OLTP_Northwind]
GO
;With CTE as (
SELECT [EmployeeID]
      ,[LastName]
      ,[FirstName]
      ,[ReportsTo]
  FROM [dbo].[Employees])

Select c1.EmployeeID as [ManagerID], c1.LastName as [Manager_LastName], c1.Firstname as [Manager_FirstName]
, c2.EmployeeID as [EmployeeID],  c2.LastName as [Employee_LastName], c2.Firstname as [Employee_FirstName]
from CTE as c1
inner join CTE as c2
on c1.EmployeeID = c2.ReportsTo